In [1]:
#we will import basic module from different libarary to scrap the wikipedia page.
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
print("Requested module imported")

Requested module imported


### Define wikipedia link to extract toronto city data

In [2]:
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
#we will use get method from requests package to extract wikipedia data and process it with beautifulsoup using lxml parser.
#sometime html page are ill defined and has error which cause exception. hence aprser is used to overcome the issue. 
# The praser modifys the ill-defined html to overcome the issue. 
page_data=requests.get(wiki_link).text
toronto_page=BeautifulSoup(page_data,'lxml')

In [4]:
# we will extract the reuired section of html page that onatian table data 
toronto_neigh=toronto_page.find('table',class_='wikitable sortable')

In [5]:
# we need to define a dataframe to insert value extracted from website
df_toro_neigh=pd.DataFrame(columns={'PostalCode','Neighborhood','Borough'})
df_toro_neigh=df_toro_neigh[['PostalCode','Borough','Neighborhood']]
df_toro_neigh

,PostalCode,Borough,Neighborhood


In [6]:
#this will extarct all the required coulmn values from the html page and assigin it to proper columns of data frame
for row_index,table_row in enumerate(toronto_neigh.find_all('tr')):
    neigh_list=[]
    for column_index,table_column in zip(df_toro_neigh.columns,table_row.find_all('td')):
        df_toro_neigh.loc[row_index,column_index]=table_column.text.split("\n")[0]
       

In [7]:
#preprocessing as required by the question set
for i in df_toro_neigh.index:
    if df_toro_neigh.loc[i,'Neighborhood'] == 'Not assigned':
        df_toro_neigh.loc[i,'Neighborhood']=df_toro_neigh.loc[i,'Borough']
    if df_toro_neigh.loc[i,'Borough'] == 'Not assigned':
        df_toro_neigh.drop([i], inplace=True)
df_toro_neigh.head()
    

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [8]:
#resetting index after removing Borough haveing Not assigned value in it
df_toro_neigh.reset_index(inplace=True)
df_toro_neigh.drop(['index'],axis=1,inplace=True)
df_toro_neigh.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
#pre-procerssing for updating all neighborhood for each PostalCode
column_index= 'Neighborhood'
for df_index,postalcode in enumerate(np.asarray(df_toro_neigh['PostalCode'])):
    merged_neighborhood=''
    for row_index,post_all in enumerate(np.asarray(df_toro_neigh['PostalCode'])):
        if postalcode == post_all:
            if merged_neighborhood == '':
                merged_neighborhood= np.asarray(df_toro_neigh['Neighborhood'])[row_index]
            else:
                merged_neighborhood = merged_neighborhood + ',' + np.asarray(df_toro_neigh['Neighborhood'])[row_index]
        
    df_toro_neigh.loc[df_index,column_index]=merged_neighborhood
df_toro_neigh.head() 

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,"Harbourfront,Regent Park,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"


In [10]:
#finally grouping them to extarct one record with all data for each postal code.
df_toro_neigh= df_toro_neigh.groupby('PostalCode',as_index=False).agg({'Borough': 'first','Neighborhood': 'first'})

In [11]:
df_toro_neigh.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
#to find out size of unique postal code in toronto
df_toro_neigh.shape

(103, 3)

In [13]:
#inorder to import latitude and longitude of given postal code of toronto, we will import geocoder and try extractung the latitude and longitutde
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim
print('Geocoder imported')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.87 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00  35.59 MB/s
Geocoder imported


In [14]:
# Since we google geocode isn't resonding properly we will use csv file to import lat and long 
csv_source='http://cocl.us/Geospatial_data'

In [15]:
df_geolocation=pd.read_csv(csv_source)
df_geolocation.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_toronto_city_geolocation=pd.concat([df_toro_neigh,df_geolocation],axis=1)

In [17]:
df_toronto_city_geolocation.drop(['Postal Code'],axis=1,inplace=True)

In [18]:
df_toronto_city_geolocation.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  55.12 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.23 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.82 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  47.77 MB/s


### Create a map of toronto city with neighborhoods superimposed on top.

In [20]:
Scarborough_data = df_toronto_city_geolocation[df_toronto_city_geolocation['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [21]:
address = 'Scarborough, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [22]:
# create map of toronto city using latitude and longitude values
Scarborough_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Borough'], Scarborough_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Scarborough_map) 
Scarborough_map

#### Define Foursquare Credentials and Version

In [38]:
# The code was removed by Watson Studio for sharing.

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )
Scarborough_venues.shape

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West,Steeles West
Upper Rouge


(84, 7)

In [26]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(84, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [27]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant,Women's Store
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant,Women's Store
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.25,0.00,0.0,0.000000,0.000000,0.000000,0.0
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,...,0.333333,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0
2,"Birch Cliff,Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.25,...,0.000000,0.000000,0.000000,0.25,0.00,0.0,0.000000,0.000000,0.000000,0.0
3,Cedarbrae,0.0,0.142857,0.142857,0.142857,0.0,0.000000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.142857,0.000000,0.000000,0.0
4,"Clairlea,Golden Mile,Oakridge",0.0,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.1,0.000000,0.000000,0.000000,0.0
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.125000,0.000000,0.000000,0.0
6,"Cliffcrest,Cliffside,Scarborough Village West",0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0
7,"Dorset Park,Scarborough Town Centre,Wexford He...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.142857,0.0
8,"East Birchmount Park,Ionview,Kennedy Park",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.142857,0.000000,0.0
9,"Guildwood,Morningside,West Hill",0.0,0.000000,0.000000,0.000000,0.0,0.166667,0.0,0.0,0.00,...,0.000000,0.166667,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0


In [29]:
num_top_venues = 3

for hood in Scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0  Breakfast Spot  0.25
1          Lounge  0.25
2    Skating Rink  0.25


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
         venue  freq
0         Park  0.33
1   Playground  0.33
2  Coffee Shop  0.33


----Birch Cliff,Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1                   Café  0.25
2  General Entertainment  0.25


----Cedarbrae----
                venue  freq
0              Bakery  0.14
1                Bank  0.14
2  Athletics & Sports  0.14


----Clairlea,Golden Mile,Oakridge----
          venue  freq
0        Bakery   0.2
1      Bus Line   0.2
2  Intersection   0.1


----Clarks Corners,Sullivan,Tam O'Shanter----
                 venue  freq
0          Pizza Place  0.25
1         Noodle House  0.12
2  Fried Chicken Joint  0.12


----Cliffcrest,Cliffside,Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1                Motel   0.5
2         

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Women's Store
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Coffee Shop,Playground,Park,Women's Store,Grocery Store
2,"Birch Cliff,Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Women's Store
3,Cedarbrae,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank
4,"Clairlea,Golden Mile,Oakridge",Bus Line,Bakery,Bus Station,Park,Fast Food Restaurant
5,"Clarks Corners,Sullivan,Tam O'Shanter",Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Italian Restaurant
6,"Cliffcrest,Cliffside,Scarborough Village West",American Restaurant,Motel,College Stadium,Grocery Store,Golf Course
7,"Dorset Park,Scarborough Town Centre,Wexford He...",Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Light Rail Station,Chinese Restaurant
8,"East Birchmount Park,Ionview,Kennedy Park",Discount Store,Train Station,Department Store,Convenience Store,Hobby Shop
9,"Guildwood,Morningside,West Hill",Breakfast Spot,Electronics Store,Rental Car Location,Pizza Place,Medical Center


In [32]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [33]:
# set number of clusters
kclusters = 3

Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 


array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2], dtype=int32)

In [34]:
Scarborough_merged = Scarborough_data.drop([16])

# add clustering labels
Scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Women's Store,College Stadium,Hakka Restaurant,Grocery Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2,Golf Course,Bar,Women's Store,College Stadium,Hakka Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0,Breakfast Spot,Electronics Store,Rental Car Location,Pizza Place,Medical Center
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Pharmacy,College Stadium,Grocery Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank


In [35]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighborhood'], Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters